In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import accuracy_score

In [3]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv'

In [4]:
!wget $data -O data-week-3.csv

--2023-10-14 07:34:09--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 977501 (955K) [text/plain]
Saving to: ‘data-week-3.csv’

data-week-3.csv     100%[===================>] 954.59K  --.-KB/s    in 0.08s   

2023-10-14 07:34:09 (12.4 MB/s) - ‘data-week-3.csv’ saved [977501/977501]



In [5]:
df = pd.read_csv('data-week-3.csv')

df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')
df.totalcharges = df.totalcharges.fillna(0)

df.churn = (df.churn == 'yes').astype(int)

In [6]:
df_full_train, df_test = train_test_split(df, test_size=0.25, random_state=1)

In [7]:
numerical = ['tenure', 'monthlycharges', 'totalcharges']

categorical = [
    'gender',
    'seniorcitizen',
    'partner',
    'dependents',
    'phoneservice',
    'multiplelines',
    'internetservice',
    'onlinesecurity',
    'onlinebackup',
    'deviceprotection',
    'techsupport',
    'streamingtv',
    'streamingmovies',
    'contract',
    'paperlessbilling',
    'paymentmethod',
]

In [8]:
def train(df_train, y_train, C=1.0):
    dicts = df_train[categorical + numerical].to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)

    model = LogisticRegression(C=C, max_iter=1000)
    model.fit(X_train, y_train)

    return dv, model

In [9]:
def predict(df, dv, model):
    dicts = df[categorical + numerical].to_dict(orient='records')

    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:, 1]

    return y_pred

In [10]:
C = 1.0
n_splits = 5

In [11]:
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

scores = []

for train_idx, val_idx in kfold.split(df_full_train):
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]

    y_train = df_train.churn.values
    y_val = df_val.churn.values

    dv, model = train(df_train, y_train, C=C)
    y_pred = predict(df_val, dv, model)

    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)

print('C=%s %.3f +- %.3f' % (C, np.mean(scores), np.std(scores)))

C=1.0 0.839 +- 0.011


In [12]:
scores

[0.8419622135350024,
 0.8568513338216217,
 0.8406761625627719,
 0.8239829091584618,
 0.8308574879227053]

In [13]:
dv, model = train(df_full_train, df_full_train.churn.values, C=1.0)
y_pred = predict(df_test, dv, model)
y_test = df_test.churn.values
auc = roc_auc_score(y_test, y_pred)
auc

0.856455259255658

**Question 1** What's the version of pipenv you installed?

In [1]:
!pipenv install

Installing dependencies from Pipfile.lock (1bed02)...
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.


In [7]:
!pipenv shell

Launching subshell in virtual environment...
/content#  . /root/.local/share/virtualenvs/content-cQIIIOO2/bin/activate
(content) /content# 
Aborted!
^C


In [6]:
!pipenv --version

pipenv, version 2023.10.3


**Question 2** What's the first hash for scikit-learn you get in Pipfile.lock?

In [8]:
!pipenv install scikit-learn==1.3.1

Installing scikit-learn==1.3.1...
Resolving scikit-learn==1.3.1...
✔ Installation Succeeded
⠋ Installing scikit-learn...
Installing dependencies from Pipfile.lock (1bed02)...
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.


In [10]:
with open('Pipfile.lock', 'r') as file:
    pipfile_lock = file.read()

import re

match = re.search(r'"scikit-learn":\s{\n\s\s"hash":\s"(.*?)"', pipfile_lock)
if match:
    scikit_learn_hash = match.group(1)
    print(scikit_learn_hash)
else:
    print("Хэш для scikit-learn не найден в Pipfile.lock")


Хэш для scikit-learn не найден в Pipfile.lock


In [12]:
!pipenv graph | grep scikit-learn


scikit-learn==1.3.1


**Save the model**

In [14]:
import pickle

In [15]:
out_file = f'model_C={C}.bin'
out_file

'model_C=1.0.bin'

In [16]:
f_out = open(out_file, 'wb')
(pickle.dump((dv, model), f_out))
f_out.close()

In [17]:
with open(out_file, 'wb') as f_out:
  (pickle.dump((dv, model), f_out))

**Load the model**

In [20]:
import pickle

In [21]:
model_file = 'model_C=1.0.bin'

In [22]:
with open(model_file, 'rb') as f_in:
  dv, model = pickle.load(f_in)

In [23]:
dv, model

(DictVectorizer(sparse=False), LogisticRegression(max_iter=1000))

In [24]:
customer = {
  'gender': 'female',
 'seniorcitizen': 0,
 'partner': 'yes',
 'dependents': 'yes',
 'phoneservice': 'yes',
 'multiplelines': 'yes',
 'internetservice': 'fiber_optic',
 'onlinesecurity': 'yes',
 'onlinebackup': 'no',
 'deviceprotection': 'yes',
 'techsupport': 'no',
 'streamingtv': 'no',
 'streamingmovies': 'no',
 'contract': 'month-to-month',
 'paperlessbilling': 'yes',
 'paymentmethod': 'electronic_check',
 'tenure': 1,
 'monthlycharges': 29.85,
 'totalcharges': 29.85
  }